## Import Modules

In [1]:
# Set paths
import os
from imp import reload

# Data manipulation
import pandas as pd
import numpy as np
from scipy import stats

# Custom package for data preprocessing
import mytools as mt 

# Set notebook options
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 10000)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Pretty display of multiple functions in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### File Location

In [2]:
# Data paths
import filepaths

### Create Project Directories and Sub-Directories

In [3]:
mt.create_directories(filepaths.project_parent_directory)

Directory  deliverables  already exists
Directory  deprecated  already exists
Directory  dictionary  already exists
Directory  visualizations  already exists
Subdirectory  data/raw  already exists
Subdirectory  data/interim  already exists
Subdirectory  data/external  already exists
Subdirectory  data/processed  already exists


## Load Data

In [4]:
df_customer = pd.read_csv(filepaths.raw_customer_data, sep=',')

df_transactions = pd.read_csv(filepaths.raw_transactions_data, sep=',')

df_products = pd.read_csv(filepaths.raw_products_data, sep=',')

## Standardize Feature Names

## Data Audit + Cleaning
### Customer Data

In [7]:
df_customer.columns = [c.lower() for c in df_customer.columns]
df_customer.shape
mt.check_unique_no(df_customer, ['customer_id'])
mt.missing_data_table(df_customer)
df_customer.head()

(5647, 4)

Data has 5647 unique customer_id
Missing data distribution:

  Variable  Count  Proportion
    gender      2        0.00
 city_code      2        0.00


,customer_id,dob,gender,city_code
0,268408,02-01-1970,M,4.00
1,269696,07-01-1970,F,8.00
2,268159,08-01-1970,F,8.00
3,270181,10-01-1970,F,2.00
4,268073,11-01-1970,M,1.00


### Transactions Data

In [8]:
df_transactions.rename(columns={'transaction_id':'trans_id', 'cust_id':'customer_id', 'tran_date':'trans_date'}, inplace=True)
df_transactions.columns = [c.lower() for c in df_transactions.columns]
df_transactions.shape
mt.check_unique_no(df_transactions, ['customer_id', 'trans_id'])
mt.missing_data_table(df_transactions)
df_transactions.head()
df_transactions.describe()

(23053, 10)

Data has 5506 unique customer_id
Data has 20878 unique trans_id
There is no missing data


,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
0,80712190438,270351,28-02-2014,1,1,-5,-772,405.30,-4265.30,e-Shop
1,29258453508,270384,27-02-2014,5,3,-5,-1497,785.92,-8270.92,e-Shop
2,51750724947,273420,24-02-2014,6,5,-2,-791,166.11,-1748.11,TeleShop
3,93274880719,271509,24-02-2014,11,6,-3,-1363,429.35,-4518.35,e-Shop
4,51750724947,273420,23-02-2014,6,5,-2,-791,166.11,-1748.11,TeleShop


,trans_id,customer_id,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt
count,23053.00,23053.00,23053.00,23053.00,23053.00,23053.00,23053.00,23053.00
mean,50073480358.45,271021.75,6.15,3.76,2.43,636.37,248.67,2107.31
std,28981936062.00,2431.69,3.73,1.68,2.27,622.36,187.18,2507.56
min,3268991.00,266783.00,1.00,1.00,-5.00,-1499.00,7.35,-8270.92
25%,24938639453.00,268935.00,3.00,2.00,1.00,312.00,98.28,762.45
50%,50093131361.00,270980.00,5.00,4.00,3.00,710.00,199.08,1754.74
75%,75329995679.00,273114.00,10.00,5.00,4.00,1109.00,365.71,3569.15
max,99987549630.00,275265.00,12.00,6.00,5.00,1500.00,787.50,8287.50


In [9]:
df_transactions['trans_id'].value_counts(dropna=False).head()

426787191      4
4170892941     4
32263938079    4
3130889793     3
42255136382    3
Name: trans_id, dtype: int64

In [10]:
df_transactions[df_transactions['trans_id'] == 426787191]

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
12961,426787191,273106,28-05-2012,1,2,-5,-109,57.23,-602.23,e-Shop
12967,426787191,273106,28-05-2012,1,2,-5,-109,57.23,-602.23,e-Shop
13156,426787191,273106,19-05-2012,1,2,-5,-109,57.23,-602.23,e-Shop
13168,426787191,273106,18-05-2012,1,2,5,109,57.23,602.23,e-Shop


In [11]:
df_transactions[df_transactions['trans_id'] == 4170892941]

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
16207,4170892941,266852,24-12-2011,8,3,-1,-412,43.26,-455.26,MBR
16267,4170892941,266852,21-12-2011,8,3,-1,-412,43.26,-455.26,MBR
16269,4170892941,266852,21-12-2011,8,3,-1,-412,43.26,-455.26,MBR
16288,4170892941,266852,20-12-2011,8,3,1,412,43.26,455.26,MBR


In [12]:
df_transactions[df_transactions['trans_id'] == 25890929042]

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
18114,25890929042,266783,24-09-2011,1,2,-4,-1321,554.82,-5838.82,e-Shop
18130,25890929042,266783,23-09-2011,1,2,4,1321,554.82,5838.82,e-Shop


In [13]:
df_transactions['trans_id'].value_counts(dropna=False)[df_transactions['trans_id'].value_counts(dropna=False).values > 1].index.nunique()

2057

In [14]:
returned_item = df_transactions['trans_id'].value_counts(dropna=False)[df_transactions['trans_id'].value_counts(dropna=False).values > 1].index.unique();
len(returned_item)

2057

In [15]:
df_transactions['returned_item'] = np.where(df_transactions['trans_id'].isin(returned_item), 'yes', 'no')

In [16]:
df_transactions['drop_record'] = np.where((df_transactions['qty'] < 0), 'yes', 'no')

In [17]:
df_transactions.head(2)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record
0,80712190438,270351,28-02-2014,1,1,-5,-772,405.30,-4265.30,e-Shop,yes,yes
1,29258453508,270384,27-02-2014,5,3,-5,-1497,785.92,-8270.92,e-Shop,yes,yes


In [18]:
df_transactions[df_transactions['qty'] < 0].shape
df_transactions[df_transactions['qty'] < 0].head()
df_transactions[df_transactions['qty'] < 0]['store_type'].value_counts(dropna=False)

(2177, 12)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record
0,80712190438,270351,28-02-2014,1,1,-5,-772,405.30,-4265.30,e-Shop,yes,yes
1,29258453508,270384,27-02-2014,5,3,-5,-1497,785.92,-8270.92,e-Shop,yes,yes
2,51750724947,273420,24-02-2014,6,5,-2,-791,166.11,-1748.11,TeleShop,yes,yes
3,93274880719,271509,24-02-2014,11,6,-3,-1363,429.35,-4518.35,e-Shop,yes,yes
4,51750724947,273420,23-02-2014,6,5,-2,-791,166.11,-1748.11,TeleShop,yes,yes


e-Shop            882
MBR               451
Flagship store    432
TeleShop          412
Name: store_type, dtype: int64

In [19]:
df_transactions[df_transactions['qty'] > 0].shape
df_transactions[df_transactions['qty'] > 0].head()
df_transactions[df_transactions['qty'] > 0]['store_type'].value_counts(dropna=False)

(20876, 12)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record
10,29258453508,270384,20-02-2014,5,3,5,1497,785.92,8270.92,e-Shop,yes,no
11,25455265351,267750,20-02-2014,12,6,3,1360,428.40,4508.40,e-Shop,no,no
12,1571002198,275023,20-02-2014,6,5,4,587,246.54,2594.54,e-Shop,no,no
14,36554696014,269345,20-02-2014,3,5,3,1253,394.69,4153.69,e-Shop,no,no
15,56814940239,268799,20-02-2014,7,5,5,368,193.20,2033.20,e-Shop,no,no


e-Shop            8429
MBR               4210
Flagship store    4145
TeleShop          4092
Name: store_type, dtype: int64

In [20]:
df_products.rename(columns={'prod_sub_cat_code':'prod_subcat_code'}, inplace=True)
df_products['counter'] = 1
mt.missing_data_table(df_products)
df_products.head()

There is no missing data


,prod_cat_code,prod_cat,prod_subcat_code,prod_subcat,counter
0,1,Clothing,4,Mens,1
1,1,Clothing,1,Women,1
2,1,Clothing,3,Kids,1
3,2,Footwear,1,Mens,1
4,2,Footwear,3,Women,1


In [21]:
prod_cat_dict = dict(zip(df_products['prod_cat_code'],df_products['prod_cat']))
prod_subcat_dict = dict(zip(df_products['prod_subcat_code'],df_products['prod_subcat']))

In [22]:
df_products.groupby(['prod_cat'])['prod_subcat'].value_counts(dropna=False)

prod_cat          prod_subcat        
Bags              Mens                   1
                  Women                  1
Books             Academic               1
                  Children               1
                  Comics                 1
                  DIY                    1
                  Fiction                1
                  Non-Fiction            1
Clothing          Kids                   1
                  Mens                   1
                  Women                  1
Electronics       Audio and video        1
                  Cameras                1
                  Computers              1
                  Mobiles                1
                  Personal Appliances    1
Footwear          Kids                   1
                  Mens                   1
                  Women                  1
Home and kitchen  Bath                   1
                  Furnishing             1
                  Kitchen                1
                

In [23]:
df_products['prod_cat'].value_counts(dropna=False)

Books               6
Electronics         5
Home and kitchen    4
Footwear            3
Clothing            3
Bags                2
Name: prod_cat, dtype: int64

In [24]:
df_products['prod_subcat'].value_counts(dropna=False)

Women                  3
Mens                   3
Kids                   2
Bath                   1
Computers              1
Audio and video        1
Furnishing             1
Tools                  1
DIY                    1
Children               1
Academic               1
Mobiles                1
Kitchen                1
Comics                 1
Non-Fiction            1
Fiction                1
Personal Appliances    1
Cameras                1
Name: prod_subcat, dtype: int64

### Transactions Data

In [25]:
converted_customers = df_transactions['customer_id'].unique().tolist()

In [26]:
df_transactions['trans_date'] = pd.to_datetime(df_transactions['trans_date'])
df_transactions['trans_date'].sort_values().min()
df_transactions['trans_date'].sort_values().max()

Timestamp('2011-01-02 00:00:00')

Timestamp('2014-12-02 00:00:00')

In [27]:
df_transactions = pd.merge(df_transactions, df_products, on=['prod_cat_code', 'prod_subcat_code'], how='left')
df_transactions.head()

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter
0,80712190438,270351,2014-02-28,1,1,-5,-772,405.30,-4265.30,e-Shop,yes,yes,Clothing,Women,1
1,29258453508,270384,2014-02-27,5,3,-5,-1497,785.92,-8270.92,e-Shop,yes,yes,Electronics,Computers,1
2,51750724947,273420,2014-02-24,6,5,-2,-791,166.11,-1748.11,TeleShop,yes,yes,Books,DIY,1
3,93274880719,271509,2014-02-24,11,6,-3,-1363,429.35,-4518.35,e-Shop,yes,yes,Home and kitchen,Bath,1
4,51750724947,273420,2014-02-23,6,5,-2,-791,166.11,-1748.11,TeleShop,yes,yes,Books,DIY,1


In [28]:
# df_transactions['qty'] = np.where((df_transactions['qty'] < 0), (-1*df_transactions['qty']), df_transactions['qty'])
# df_transactions['rate'] = np.where((df_transactions['rate'] < 0), (-1*df_transactions['rate']), df_transactions['rate'])
# df_transactions['total_amt'] = np.where((df_transactions['total_amt'] < 0), (-1*df_transactions['total_amt']), df_transactions['total_amt'])
# df_transactions.head()

In [29]:
df_transactions['store_type'].value_counts(dropna=False)

e-Shop            9311
MBR               4661
Flagship store    4577
TeleShop          4504
Name: store_type, dtype: int64

In [30]:
df_transactions = df_transactions.sort_values(['customer_id','trans_date'])

In [31]:
df_transactions['assessment_date'] = df_transactions['trans_date'].sort_values().max()

In [32]:
df_transactions['duration'] = df_transactions.groupby(['customer_id'])['trans_date'].transform(pd.Series.diff).shift(-1)
df_transactions['duration'] = df_transactions.apply(lambda r: r['assessment_date'] - r['trans_date'] if pd.isnull(r['duration']) else r['duration'], axis=1)
df_transactions['duration'] = (df_transactions['duration']/np.timedelta64(1, 'D'))

In [33]:
df_transactions.head(7)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration
18130,25890929042,266783,2011-09-23,1,2,4,1321,554.82,5838.82,e-Shop,yes,no,Footwear,Mens,1,2014-12-02,1.00
18114,25890929042,266783,2011-09-24,1,2,-4,-1321,554.82,-5838.82,e-Shop,yes,yes,Footwear,Mens,1,2014-12-02,393.00
10030,98477711300,266783,2012-10-21,4,1,3,93,29.30,308.30,TeleShop,no,no,Clothing,Mens,1,2014-12-02,122.00
7515,8410316370,266783,2013-02-20,4,1,1,869,91.25,960.25,e-Shop,no,no,Clothing,Mens,1,2014-12-02,194.00
7722,16999552161,266783,2013-09-02,10,5,2,835,175.35,1845.35,e-Shop,no,no,Books,Non-Fiction,1,2014-12-02,456.00
9055,36310127403,266784,2012-04-12,4,3,2,200,42.00,442.00,Flagship store,no,no,Electronics,Mobiles,1,2014-12-02,133.00
11286,54234600611,266784,2012-08-23,10,5,3,1291,406.67,4279.66,TeleShop,no,no,Books,Non-Fiction,1,2014-12-02,17.00


In [34]:
df_trans_overall = df_transactions.groupby(['customer_id'], as_index=False).agg({'trans_date':['first', 'last', pd.Series.nunique],
                                                                                 'duration': ['min', 'max', 'mean', 'last'],
                                                        'trans_id':'count',
                                                        'qty':['min', 'max', 'sum'],
                                                       'tax':'sum',
                                                       'total_amt':'sum' }) #, lambda x: stats.mode(x)[0][0]]|, 'prod_cat':[pd.Series.nunique],
#                                                          'prod_subcat':[pd.Series.nunique],
#                                                        'store_type':[pd.Series.nunique]

In [35]:
df_trans_overall.shape
df_trans_overall.columns = [".".join(x).strip('.') for x in df_trans_overall.columns.ravel()] 
df_trans_overall.rename(columns={'trans_date.first':'conversion_date', 'trans_date.last':'last_purchase_date',
                                'trans_date.nunique':'number_of_purchase_days'}, inplace=True)
mt.check_unique_no(df_trans_overall, ['customer_id'])
df_trans_overall.head()

(5506, 14)

Data has 5506 unique customer_id


,customer_id,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum
0,266783,2011-09-23,2013-09-02,5,1.00,456.00,233.20,456.00,5,-4,4,6,1405.53,3113.89
1,266784,2012-04-12,2012-09-09,3,17.00,814.00,321.33,814.00,3,2,5,10,541.07,5694.06
2,266785,2011-03-15,2013-02-13,8,5.00,657.00,169.75,657.00,8,-5,5,23,3351.60,21613.80
3,266788,2011-09-13,2013-12-02,4,84.00,397.00,294.00,365.00,4,1,4,8,578.97,6092.97
4,266794,2011-03-18,2014-12-02,11,0.00,520.00,112.92,0.00,12,-1,4,30,2684.74,27981.92


In [36]:
df_trans_overall['assessment_date'] = df_transactions['trans_date'].sort_values().max()

In [37]:
df_trans_overall['customer_age.days'] = (df_trans_overall['assessment_date'] - df_trans_overall['conversion_date'])/np.timedelta64(1,'D')

In [38]:
df_trans_overall['customer_age.years'] = (df_trans_overall['assessment_date'] - df_trans_overall['conversion_date'])/np.timedelta64(1,'Y')

In [39]:
age_bins =  [0, 1, 2, 3, 4]
labels = ['_<01','01_02', '02_03', '03_04']
df_trans_overall['customer_age.years.group'] = pd.cut(df_trans_overall['customer_age.years'], age_bins, labels = labels,include_lowest = True)

In [40]:
# def customer_profile(_df, unique_id, value, featurelist):
#     df_list = []
#     for feature in featurelist:
#         _df[feature] = _df[feature].astype(str)
#         _df_temp = pd.crosstab(_df[unique_id], _df[feature], values=_df[value], 
#                                aggfunc=['count','sum', 'mean'], dropna=False).fillna(0).reset_index()
#         _df_temp.columns = [".".join(x).strip('.') for x in _df_temp.columns.ravel()] 
#         df_list.append(_df_temp)
#         dfs = [df.set_index(unique_id) for df in df_list]
#         df = pd.concat(dfs, axis=1)
#         df.reset_index(inplace=True)
#     return df

In [41]:
# df_product_cat = customer_profile(df_transactions, 'customer_id', 'total_amt', ['prod_cat'])
# df_product_cat.head()

In [42]:
# df_product_subcat = customer_profile(df_transactions, 'customer_id', 'total_amt', ['prod_subcat'])
# df_product_subcat.head()

In [43]:
# df_stores = customer_profile(df_transactions, 'customer_id', 'total_amt', ['store_type'])
# df_stores.head()

In [44]:
# df_trans_overall = pd.merge(df_trans_overall, df_stores, on=['customer_id'], how='left')
# df_trans_overall.head()

In [45]:
# df_trans_overall['Flagship_store_spend.prop'] = df_trans_overall['sum.Flagship store']/df_trans_overall['total_amt.sum']
# df_trans_overall['MBR_spend.prop'] = df_trans_overall['sum.MBR']/df_trans_overall['total_amt.sum']
# df_trans_overall['TeleShop_spend.prop'] = df_trans_overall['sum.TeleShop']/df_trans_overall['total_amt.sum']
# df_trans_overall['e-Shop.prop'] = df_trans_overall['sum.e-Shop']/df_trans_overall['total_amt.sum']

In [46]:
# df_trans_overall = pd.merge(df_trans_overall, df_product_cat, on=['customer_id'], how='left')
# df_trans_overall.head()

In [47]:
# df_trans_overall['Bags.prop'] = df_trans_overall['sum.Bags']/df_trans_overall['total_amt.sum']
# df_trans_overall['Books.prop'] = df_trans_overall['sum.Books']/df_trans_overall['total_amt.sum']
# df_trans_overall['Clothing.prop'] = df_trans_overall['sum.Clothing']/df_trans_overall['total_amt.sum']
# df_trans_overall['Electronics.prop'] = df_trans_overall['sum.Electronics']/df_trans_overall['total_amt.sum']
# df_trans_overall['Footwear.prop'] = df_trans_overall['sum.Footwear']/df_trans_overall['total_amt.sum']
# df_trans_overall['Home and kitchen.prop'] = df_trans_overall['sum.Home and kitchen']/df_trans_overall['total_amt.sum']

In [48]:
# df_trans_overall = pd.merge(df_trans_overall, df_product_subcat, on=['customer_id'], how='left')
# df_trans_overall.head()

In [49]:
# df_trans_overall['Academic.prop'] = df_trans_overall['sum.Academic']/df_trans_overall['total_amt.sum']
# df_trans_overall['Audio and video.prop'] = df_trans_overall['sum.Audio and video']/df_trans_overall['total_amt.sum']
# df_trans_overall['Bath.prop'] = df_trans_overall['sum.Bath']/df_trans_overall['total_amt.sum']
# df_trans_overall['Cameras.prop'] = df_trans_overall['sum.Cameras']/df_trans_overall['total_amt.sum']
# df_trans_overall['Children.prop'] = df_trans_overall['sum.Children']/df_trans_overall['total_amt.sum']
# df_trans_overall['Comics.prop'] = df_trans_overall['sum.Comics']/df_trans_overall['total_amt.sum']
# df_trans_overall['Computers.prop'] = df_trans_overall['sum.Computers']/df_trans_overall['total_amt.sum']
# df_trans_overall['DIY.prop'] = df_trans_overall['sum.DIY']/df_trans_overall['total_amt.sum']
# df_trans_overall['Fiction.prop'] = df_trans_overall['sum.Fiction']/df_trans_overall['total_amt.sum']
# df_trans_overall['Furnishing.prop'] = df_trans_overall['sum.Furnishing']/df_trans_overall['total_amt.sum']
# df_trans_overall['Kids.prop'] = df_trans_overall['sum.Kids']/df_trans_overall['total_amt.sum']
# df_trans_overall['Kitchen.prop'] = df_trans_overall['sum.Kitchen']/df_trans_overall['total_amt.sum']
# df_trans_overall['Mens.prop'] = df_trans_overall['sum.Mens']/df_trans_overall['total_amt.sum']
# df_trans_overall['Mobiles.prop'] = df_trans_overall['sum.Mobiles']/df_trans_overall['total_amt.sum']
# df_trans_overall['Non-Fiction.prop'] = df_trans_overall['sum.Non-Fiction']/df_trans_overall['total_amt.sum']
# df_trans_overall['Personal Appliances.prop'] = df_trans_overall['sum.Personal Appliances']/df_trans_overall['total_amt.sum']
# df_trans_overall['Tools.prop'] = df_trans_overall['sum.Tools']/df_trans_overall['total_amt.sum']
# df_trans_overall['Women.prop'] = df_trans_overall['sum.Women']/df_trans_overall['total_amt.sum']

In [50]:
df_trans_overall.head()

,customer_id,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum,assessment_date,customer_age.days,customer_age.years,customer_age.years.group
0,266783,2011-09-23,2013-09-02,5,1.00,456.00,233.20,456.00,5,-4,4,6,1405.53,3113.89,2014-12-02,1166.00,3.19,03_04
1,266784,2012-04-12,2012-09-09,3,17.00,814.00,321.33,814.00,3,2,5,10,541.07,5694.06,2014-12-02,964.00,2.64,02_03
2,266785,2011-03-15,2013-02-13,8,5.00,657.00,169.75,657.00,8,-5,5,23,3351.60,21613.80,2014-12-02,1358.00,3.72,03_04
3,266788,2011-09-13,2013-12-02,4,84.00,397.00,294.00,365.00,4,1,4,8,578.97,6092.97,2014-12-02,1176.00,3.22,03_04
4,266794,2011-03-18,2014-12-02,11,0.00,520.00,112.92,0.00,12,-1,4,30,2684.74,27981.92,2014-12-02,1355.00,3.71,03_04


### Customer Data

In [51]:
df_customer['customer_status'] = np.where((df_customer['customer_id'].isin(converted_customers)), 'converted', 'voluntarily churned - not converted')
df_customer['customer_status'].value_counts(dropna=False)

converted                              5506
voluntarily churned - not converted     141
Name: customer_status, dtype: int64

In [52]:
df_customer['dob'] = pd.to_datetime(df_customer['dob'])
df_customer['dob'].sort_values().min()
df_customer['dob'].sort_values().max()

Timestamp('1970-01-02 00:00:00')

Timestamp('1992-12-29 00:00:00')

In [53]:
df_customer.tail()

,customer_id,dob,gender,city_code,customer_status
5642,274474,1992-12-19,M,2.00,converted
5643,267666,1992-12-24,M,6.00,converted
5644,270476,1992-12-25,F,3.00,converted
5645,269626,1992-12-27,F,5.00,converted
5646,274308,1992-12-29,F,5.00,converted


In [54]:
df_transactions['trans_date.day'] = df_transactions['trans_date'].dt.day
df_transactions['trans_date.month_num'] = df_transactions['trans_date'].dt.month.map("{:02}".format)
df_transactions['trans_date.year'] = df_transactions['trans_date'].dt.year
df_transactions['trans_date.year_month'] = df_transactions['trans_date'].dt.year.map(str) + '_' + df_transactions['trans_date'].dt.month.map("{:02}".format)
df_transactions['trans_date.hour'] = df_transactions['trans_date'].dt.hour
df_transactions['trans_date.weekday'] = df_transactions['trans_date'].dt.day_name()
df_transactions['trans_date.week_of_year'] = df_transactions['trans_date'].dt.week.map("{:02}".format)

In [55]:
df_transactions['trans_date.weekday'].head()

18130       Friday
18114     Saturday
10030       Sunday
7515     Wednesday
7722        Monday
Name: trans_date.weekday, dtype: object

In [56]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_num = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_dict = dict(zip(month_num, month_name))

for k, v in month_dict.items():
    mask = df_transactions['trans_date.month_num'].str.contains(k, case=True)

    df_transactions.loc[mask,'trans_date.month'] = v

In [57]:
df_transactions['trans_date.month'] = df_transactions['trans_date.month_num'] + '_' + df_transactions['trans_date.month'].map(str)

In [58]:
df_transactions['trans_date.month'].value_counts(dropna=False)

01_Jan    2031
10_Oct    2019
09_Sep    1986
03_Mar    1972
12_Dec    1932
08_Aug    1919
07_Jul    1917
02_Feb    1888
11_Nov    1876
05_May    1853
04_Apr    1842
06_Jun    1818
Name: trans_date.month, dtype: int64

In [59]:
weekday_name = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
weekday_num = ['01', '02', '03', '04', '05', '06', '07']
weekday_dict = dict(zip(weekday_name, weekday_num))

for k, v in weekday_dict.items():
    mask = df_transactions['trans_date.weekday'].str.contains(k, case=True)

    df_transactions.loc[mask, 'trans_date.weekday_num'] = v

In [60]:
def bin_week(x):
    if ( x>= 1) and (x <= 7):
        return 'month.week1'

    elif (x >= 8) and (x <= 14):
        return 'month.week2'

    elif (x >= 15) and (x <= 21):
        return 'month.week3'

    else:
        return 'month.week4'

In [61]:
df_transactions['trans_date.week_of_month'] = df_transactions['trans_date'].dt.day.apply(bin_week)

In [62]:
df_transactions['trans_date.weekday'] = df_transactions['trans_date.weekday_num'] + '_' + df_transactions['trans_date.weekday'].map(str)

In [63]:
df_transactions['trans_date.weekday'].value_counts(dropna=False)

01_Sunday       3417
03_Tuesday      3315
05_Thursday     3305
07_Saturday     3278
06_Friday       3250
02_Monday       3245
04_Wednesday    3243
Name: trans_date.weekday, dtype: int64

In [64]:
# df_transactions['trans_date.hour'].describe()
# df_transactions['trans_date.time_of_day']

In [65]:
df_transactions = df_transactions.sort_values(['customer_id', 'trans_date'])

In [66]:
df_first_purchase_date = df_transactions.groupby(['customer_id']).agg({'trans_date':'first'})

In [67]:
df_first_purchase_date.rename(columns={'trans_date':'first_purchase_date'}, inplace=True)

In [68]:
df_first_purchase_date.shape

(5506, 1)

In [69]:
df_transactions = pd.merge(df_transactions, df_first_purchase_date, on='customer_id', how='left')

In [70]:
df_transactions['first_purchase.cohort'] = df_transactions['first_purchase_date'].dt.year.map(str) + "_" + df_transactions['first_purchase_date'].dt.month.map("{:02}".format)

In [71]:
df_transactions['purchase_date.cohort'] = df_transactions['trans_date'].dt.year.map(str) + "_" + df_transactions['trans_date'].dt.month.map("{:02}".format)

In [72]:
df_earliest_purchase_date = df_first_purchase_date.copy()
df_earliest_purchase_date.rename(columns={'first_purchase_date':'trans_date'}, inplace=True)

In [73]:
df_earliest_purchase_date['customer_type.period'] = 'new'

In [74]:
df_transactions = pd.merge(df_transactions, df_earliest_purchase_date, on=['customer_id', 'trans_date'], how='left')

In [75]:
df_transactions['customer_type.period']  = np.where(df_transactions['customer_type.period'].isnull(), 'existing', df_transactions['customer_type.period'])

In [76]:
df_transactions['qty_negative'] = np.where((df_transactions['qty'] < 0), 'yes', 'no')

In [77]:
trans_id_returned = df_transactions[df_transactions['qty_negative'] == 'yes']['trans_id'].unique().tolist()

In [78]:
df_transactions['returned'] = np.where((df_transactions['trans_id'].isin(trans_id_returned)), 'yes', 'no')

In [79]:
df_transactions.drop(['qty_negative'], axis=1, inplace=True)

In [80]:
df_transactions.head()

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,first_purchase_date,first_purchase.cohort,purchase_date.cohort,customer_type.period,returned
0,25890929042,266783,2011-09-23,1,2,4,1321,554.82,5838.82,e-Shop,yes,no,Footwear,Mens,1,2014-12-02,1.00,23,09,2011,2011_09,0,06_Friday,38,09_Sep,06,month.week4,2011-09-23,2011_09,2011_09,new,yes
1,25890929042,266783,2011-09-24,1,2,-4,-1321,554.82,-5838.82,e-Shop,yes,yes,Footwear,Mens,1,2014-12-02,393.00,24,09,2011,2011_09,0,07_Saturday,38,09_Sep,07,month.week4,2011-09-23,2011_09,2011_09,existing,yes
2,98477711300,266783,2012-10-21,4,1,3,93,29.30,308.30,TeleShop,no,no,Clothing,Mens,1,2014-12-02,122.00,21,10,2012,2012_10,0,01_Sunday,42,10_Oct,01,month.week3,2011-09-23,2011_09,2012_10,existing,no
3,8410316370,266783,2013-02-20,4,1,1,869,91.25,960.25,e-Shop,no,no,Clothing,Mens,1,2014-12-02,194.00,20,02,2013,2013_02,0,04_Wednesday,08,02_Feb,04,month.week3,2011-09-23,2011_09,2013_02,existing,no
4,16999552161,266783,2013-09-02,10,5,2,835,175.35,1845.35,e-Shop,no,no,Books,Non-Fiction,1,2014-12-02,456.00,2,09,2013,2013_09,0,02_Monday,36,09_Sep,02,month.week1,2011-09-23,2011_09,2013_09,existing,no


In [81]:
df_transactions.tail()

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,first_purchase_date,first_purchase.cohort,purchase_date.cohort,customer_type.period,returned
23048,94712826085,275264,2011-05-08,10,5,5,221,116.03,1221.03,TeleShop,no,no,Books,Non-Fiction,1,2014-12-02,94.00,8,05,2011,2011_05,0,01_Sunday,18,05_May,01,month.week2,2011-05-08,2011_05,2011_05,new,no
23049,81382444243,275264,2011-08-10,12,6,4,587,246.54,2594.54,e-Shop,no,no,Home and kitchen,Tools,1,2014-12-02,1210.00,10,08,2011,2011_08,0,04_Wednesday,32,08_Aug,04,month.week2,2011-05-08,2011_05,2011_08,existing,no
23050,7214136016,275265,2011-12-17,1,4,1,222,23.31,245.31,TeleShop,no,no,Bags,Mens,1,2014-12-02,9.00,17,12,2011,2011_12,0,07_Saturday,50,12_Dec,07,month.week3,2011-12-17,2011_12,2011_12,new,no
23051,38961184788,275265,2011-12-26,12,5,3,188,59.22,623.22,e-Shop,no,no,Books,Academic,1,2014-12-02,434.00,26,12,2011,2011_12,0,02_Monday,52,12_Dec,02,month.week4,2011-12-17,2011_12,2011_12,existing,no
23052,24113900219,275265,2013-03-04,2,6,3,719,226.49,2383.49,Flagship store,no,no,Home and kitchen,Furnishing,1,2014-12-02,638.00,4,03,2013,2013_03,0,02_Monday,10,03_Mar,02,month.week1,2011-12-17,2011_12,2013_03,existing,no


In [82]:
df_transactions[df_transactions['customer_id'] == 268624]

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,first_purchase_date,first_purchase.cohort,purchase_date.cohort,customer_type.period,returned
4849,36332303449,268624,2011-01-02,10,6,-4,-295,123.90,-1303.90,Flagship store,yes,yes,Home and kitchen,Kitchen,1,2014-12-02,25.00,2,01,2011,2011_01,0,01_Sunday,52,01_Jan,01,month.week1,2011-01-02,2011_01,2011_01,new,yes
4850,36332303449,268624,2011-01-27,10,6,4,295,123.90,1303.90,Flagship store,yes,no,Home and kitchen,Kitchen,1,2014-12-02,53.00,27,01,2011,2011_01,0,05_Thursday,04,01_Jan,05,month.week4,2011-01-02,2011_01,2011_01,existing,yes
4851,66278698494,268624,2011-03-21,1,2,1,418,43.89,461.89,TeleShop,no,no,Footwear,Mens,1,2014-12-02,186.00,21,03,2011,2011_03,0,02_Monday,12,03_Mar,02,month.week3,2011-01-02,2011_01,2011_03,existing,no
4852,89509659612,268624,2011-09-23,8,3,5,560,294.00,3094.00,MBR,no,no,Electronics,Personal Appliances,1,2014-12-02,383.00,23,09,2011,2011_09,0,06_Friday,38,09_Sep,06,month.week4,2011-01-02,2011_01,2011_09,existing,no
4853,71442479722,268624,2012-10-10,2,6,3,1477,465.25,4896.26,Flagship store,no,no,Home and kitchen,Furnishing,1,2014-12-02,168.00,10,10,2012,2012_10,0,04_Wednesday,41,10_Oct,04,month.week2,2011-01-02,2011_01,2012_10,existing,no
4854,26264839788,268624,2013-03-27,1,1,2,710,149.10,1569.10,e-Shop,no,no,Clothing,Women,1,2014-12-02,112.00,27,03,2013,2013_03,0,04_Wednesday,13,03_Mar,04,month.week4,2011-01-02,2011_01,2013_03,existing,no
4855,39847252322,268624,2013-07-17,1,2,5,1422,746.55,7856.55,Flagship store,no,no,Footwear,Mens,1,2014-12-02,63.00,17,07,2013,2013_07,0,04_Wednesday,29,07_Jul,04,month.week3,2011-01-02,2011_01,2013_07,existing,no
4856,31803201960,268624,2013-09-18,11,5,5,1437,754.42,7939.43,e-Shop,no,no,Books,Children,1,2014-12-02,440.00,18,09,2013,2013_09,0,04_Wednesday,38,09_Sep,04,month.week3,2011-01-02,2011_01,2013_09,existing,no


### Master File

In [83]:
df_master = pd.merge(df_customer, df_trans_overall,  on=['customer_id'], how='left')
# df_master['assessment_date'] = df_transactions['trans_date'].sort_values().max()
mt.check_unique_no(df_master, ['customer_id'])
df_master.head()

Data has 5647 unique customer_id


,customer_id,dob,gender,city_code,customer_status,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum,assessment_date,customer_age.days,customer_age.years,customer_age.years.group
0,268408,1970-02-01,M,4.00,converted,2011-12-07,2014-01-13,11.00,4.00,323.00,99.18,323.00,11.00,-5.00,5.00,33.00,2526.93,24272.43,2014-12-02,1091.00,2.99,02_03
1,269696,1970-07-01,F,8.00,converted,2011-09-18,2012-08-04,3.00,111.00,850.00,390.33,850.00,3.00,-4.00,4.00,3.00,1043.91,4488.51,2014-12-02,1171.00,3.21,03_04
2,268159,1970-08-01,F,8.00,converted,2012-06-02,2013-03-31,6.00,21.00,611.00,152.17,611.00,6.00,1.00,5.00,19.00,1699.85,17888.84,2014-12-02,913.00,2.50,02_03
3,270181,1970-10-01,F,2.00,converted,2011-03-18,2014-09-01,10.00,2.00,306.00,135.50,92.00,10.00,-4.00,5.00,16.00,2440.31,14869.99,2014-12-02,1355.00,3.71,03_04
4,268073,1970-11-01,M,1.00,converted,2011-11-19,2013-12-30,3.00,1.00,771.00,369.67,337.00,3.00,-1.00,5.00,5.00,896.28,7956.00,2014-12-02,1109.00,3.04,03_04


In [84]:
df_master['customer_status'].value_counts(dropna=False)

converted                              5506
voluntarily churned - not converted     141
Name: customer_status, dtype: int64

In [85]:
df_master['conversion_date.month_num'] = df_master['conversion_date'].dt.month.map("{:02}".format)

In [86]:
df_master['conversion_date.month_num'].value_counts(dropna=False)

3.0     689
2.0     648
1.0     621
4.0     568
5.0     500
6.0     453
7.0     436
8.0     387
9.0     356
10.0    334
11.0    284
12.0    230
nan     141
Name: conversion_date.month_num, dtype: int64

In [87]:
# df_master['conversion_date.month_num'] = df_master['conversion_date.month_num'].astype('O').astype('int64')

In [88]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_num = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
month_dict = dict(zip(month_num, month_name))

for k, v in month_dict.items():
    mask = df_master['conversion_date.month_num'].str.contains(k, case=True)

    df_master.loc[mask,'conversion_date.month'] = v

In [89]:
df_master['conversion_date.month'] = df_master['conversion_date.month_num'] + '_' + df_master['conversion_date.month'].map(str)

In [90]:
df_master['conversion_date.month'].value_counts(dropna=False)

3.0_Mar     689
2.0_Feb     648
1.0_Jan     621
4.0_Apr     568
5.0_May     500
6.0_Jun     453
7.0_Jul     436
8.0_Aug     387
9.0_Sep     356
10.0_Oct    334
11.0_Nov    284
12.0_Dec    230
nan_nan     141
Name: conversion_date.month, dtype: int64

In [91]:
df_master.head(1)

,customer_id,dob,gender,city_code,customer_status,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum,assessment_date,customer_age.days,customer_age.years,customer_age.years.group,conversion_date.month_num,conversion_date.month
0,268408,1970-02-01,M,4.00,converted,2011-12-07,2014-01-13,11.00,4.00,323.00,99.18,323.00,11.00,-5.00,5.00,33.00,2526.93,24272.43,2014-12-02,1091.00,2.99,02_03,12.0,12.0_Dec


In [92]:
df_master['biological_age.actual'] = (df_master['assessment_date'] - df_master['dob'])/np.timedelta64(1, 'Y')

In [93]:
df_master['biological_age.actual'].describe()

count   5506.00
mean      33.34
std        6.61
min       21.93
25%       27.52
50%       33.37
75%       39.09
max       44.92
Name: biological_age.actual, dtype: float64

In [94]:
df_master['biological_age'] = np.round(df_master['biological_age.actual'])

In [95]:
df_master['biological_age'].describe()

count   5506.00
mean      33.34
std        6.62
min       22.00
25%       28.00
50%       33.00
75%       39.00
max       45.00
Name: biological_age, dtype: float64

In [96]:
age_bins =  [21, 25, 30, 35, 40, 45]
labels = ['22_25','25_30', '30_35', '35_40', '40_45']
df_master['biological_age.group'] = pd.cut(df_master['biological_age'], age_bins, labels = labels,include_lowest = True)

In [97]:
df_master['number_of_purchase_days'].describe()

count   5506.00
mean       4.18
std        2.11
min        1.00
25%        3.00
50%        4.00
75%        6.00
max       13.00
Name: number_of_purchase_days, dtype: float64

In [98]:
df_master['number_of_purchase_days'] =df_master['number_of_purchase_days'].replace(np.nan, 0)

In [99]:
df_master['repeat_purchaser'] = np.where((df_master['number_of_purchase_days'] > 1), 'yes',
                                      np.where((df_master['number_of_purchase_days']== 0),'never purchased',
                                        'no'))

In [100]:
df_master[df_master['customer_id'] == 266783]

,customer_id,dob,gender,city_code,customer_status,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum,assessment_date,customer_age.days,customer_age.years,customer_age.years.group,conversion_date.month_num,conversion_date.month,biological_age.actual,biological_age,biological_age.group,repeat_purchaser
1019,266783,1974-01-05,M,4.00,converted,2011-09-23,2013-09-02,5.00,1.00,456.00,233.20,456.00,5.00,-4.00,4.00,6.00,1405.53,3113.89,2014-12-02,1166.00,3.19,03_04,9.0,9.0_Sep,40.91,41.00,40_45,yes


In [101]:
df_master[df_master['customer_age.years'] == df_master['customer_age.years'].min()]

,customer_id,dob,gender,city_code,customer_status,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum,assessment_date,customer_age.days,customer_age.years,customer_age.years.group,conversion_date.month_num,conversion_date.month,biological_age.actual,biological_age,biological_age.group,repeat_purchaser
3951,274213,1986-03-27,M,9.00,converted,2014-12-01,2014-12-01,1.00,1.00,1.00,1.00,1.00,1.00,2.00,2.00,2.00,279.72,2943.72,2014-12-02,1.00,0.00,_<01,12.0,12.0_Dec,28.69,29.00,25_30,no


In [102]:
df_master['1_day_amt.avg'] = (df_master['total_amt.sum']/df_master['customer_age.days']) * 1
df_master['7_day_amt.avg'] = (df_master['total_amt.sum']/df_master['customer_age.days']) * 7
df_master['30_day_amt.avg'] = (df_master['total_amt.sum']/df_master['customer_age.days']) * 30

In [103]:
df_master['1_day_num.avg'] = (df_master['qty.sum']/df_master['customer_age.days']) * 1
df_master['7_day_num.avg'] = (df_master['qty.sum']/df_master['customer_age.days']) * 7
df_master['30_day_num.avg'] = (df_master['qty.sum']/df_master['customer_age.days']) * 30

In [104]:
df_master[df_master['repeat_purchaser'] == 'yes'].head()

,customer_id,dob,gender,city_code,customer_status,conversion_date,last_purchase_date,number_of_purchase_days,duration.min,duration.max,duration.mean,duration.last,trans_id.count,qty.min,qty.max,qty.sum,tax.sum,total_amt.sum,assessment_date,customer_age.days,customer_age.years,customer_age.years.group,conversion_date.month_num,conversion_date.month,biological_age.actual,biological_age,biological_age.group,repeat_purchaser,1_day_amt.avg,7_day_amt.avg,30_day_amt.avg,1_day_num.avg,7_day_num.avg,30_day_num.avg
0,268408,1970-02-01,M,4.00,converted,2011-12-07,2014-01-13,11.00,4.00,323.00,99.18,323.00,11.00,-5.00,5.00,33.00,2526.93,24272.43,2014-12-02,1091.00,2.99,02_03,12.0,12.0_Dec,44.83,45.00,40_45,yes,22.25,155.74,667.44,0.03,0.21,0.91
1,269696,1970-07-01,F,8.00,converted,2011-09-18,2012-08-04,3.00,111.00,850.00,390.33,850.00,3.00,-4.00,4.00,3.00,1043.91,4488.51,2014-12-02,1171.00,3.21,03_04,9.0,9.0_Sep,44.42,44.00,40_45,yes,3.83,26.83,114.99,0.00,0.02,0.08
2,268159,1970-08-01,F,8.00,converted,2012-06-02,2013-03-31,6.00,21.00,611.00,152.17,611.00,6.00,1.00,5.00,19.00,1699.85,17888.84,2014-12-02,913.00,2.50,02_03,6.0,6.0_Jun,44.34,44.00,40_45,yes,19.59,137.15,587.80,0.02,0.15,0.62
3,270181,1970-10-01,F,2.00,converted,2011-03-18,2014-09-01,10.00,2.00,306.00,135.50,92.00,10.00,-4.00,5.00,16.00,2440.31,14869.99,2014-12-02,1355.00,3.71,03_04,3.0,3.0_Mar,44.17,44.00,40_45,yes,10.97,76.82,329.22,0.01,0.08,0.35
4,268073,1970-11-01,M,1.00,converted,2011-11-19,2013-12-30,3.00,1.00,771.00,369.67,337.00,3.00,-1.00,5.00,5.00,896.28,7956.00,2014-12-02,1109.00,3.04,03_04,11.0,11.0_Nov,44.09,44.00,40_45,yes,7.17,50.22,215.22,0.00,0.03,0.14


In [105]:
df_master['repeat_purchaser'].value_counts(dropna=False)

yes                5041
no                  465
never purchased     141
Name: repeat_purchaser, dtype: int64

In [106]:
df_master['qty.sum'].sum()

56074.0

In [107]:
df_master['total_amt.sum'].sum()

48579771.37

In [108]:
df_master['total_amt.sum'].sum() - df_master['tax.sum'].sum() 

42847246.599999994

In [109]:
df_transactions.to_csv(filepaths.interim_transactions_data, index=False)
df_master.to_csv(filepaths.master_file_data, index=False)